In [1]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

import matplotlib.pyplot as plt


2.1.2
2.1.2


In [2]:
pip install mir_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 897.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100701 sha256=d5eb88f0c22c93d44e98b72a15b3a6bb88fc32ddfc85f2b10af6e91cd0a1f482
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built mir_eval
Note: you may need to restart the kernel to use updated packages.


In [3]:
from IPython.display import Audio
from mir_eval import separation
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB_PLUS
from torchaudio.utils import download_asset

In [4]:
bundle = HDEMUCS_HIGH_MUSDB_PLUS

model = bundle.get_model()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

last_layer = list(model.children())[-1]  
second_last_layer = list(model.children())[-2]  

for param in model.parameters():
    param.requires_grad = False
    
for param in last_layer.parameters():
    param.requires_grad = True
for param in second_last_layer.parameters():
    param.requires_grad = True

model.to(device)
sample_rate = bundle.sample_rate

print(f"Sample rate: {sample_rate}")

100%|██████████| 319M/319M [00:01<00:00, 273MB/s]  


Sample rate: 44100


In [5]:
input_directory = '/kaggle/input/noisy-trainset-28spk-wav/noisy_trainset_28spk_wav'
target_directory = '/kaggle/input/clean-trainset-28spk-wav/clean_trainset_28spk_wav'

In [6]:
import torch
import torchaudio
import os

class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, input_directory, target_directory, target_length):
        self.input_file_paths = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith('.wav')]
        self.target_file_paths = [os.path.join(target_directory, filename) for filename in os.listdir(target_directory) if filename.endswith('.wav')]
        self.target_length = target_length
        self.resample = torchaudio.transforms.Resample(orig_freq=44100, new_freq=target_length)

    def __len__(self):
        return len(self.input_file_paths)

    def __getitem__(self, idx):
        input_file_path = self.input_file_paths[idx]
        target_file_path = self.target_file_paths[idx % len(self.target_file_paths)]

        input_waveform, _ = torchaudio.load(input_file_path)
        input_waveform = self.resample(input_waveform)

        target_waveform, _ = torchaudio.load(target_file_path)
        target_waveform = self.resample(target_waveform)

        input_waveform = self.prepare_waveform(input_waveform)
        target_waveform = self.prepare_waveform(target_waveform)

        input_waveform_dual = torch.stack([input_waveform, input_waveform])
        target_waveform_dual = torch.stack([target_waveform, target_waveform])

        return input_waveform_dual, target_waveform_dual

    def prepare_waveform(self, waveform):
        if waveform.size(1) < self.target_length:
            waveform = torch.nn.functional.pad(waveform, (0, self.target_length - waveform.size(1)))
        elif waveform.size(1) > self.target_length:
            waveform = waveform[:, :self.target_length]
        return waveform



target_length = 16000 
 

dataset = AudioDataset(input_directory, target_directory, target_length)



In [7]:
target_length = 160000
batch_size = 16
num_epochs = 10


dataset = AudioDataset(input_directory, target_directory, target_length=target_length)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [8]:
torch.cuda.empty_cache()

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchaudio.transforms import Resample


loss_function = nn.MSELoss()
num_epochs = 10
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for input_batch, target_batch in data_loader:
        input_batch = input_batch.squeeze(2)
        input_batch = input_batch.to(device)
        target_batch = target_batch.squeeze(2)
        target_batch = target_batch.to(device)

        output_batch = model(input_batch)
        output_batch = output_batch[2, :, :,:]


#         loss = loss_function(output_batch, target_batch)

        sdr_loss = 0.0
        for i in range(output_batch.shape[0]):
            reference_sources = target_batch[i].cpu().numpy()
            estimated_sources = output_batch[i].detach().cpu().numpy()
#             estimated_sources = estimated_sources[2, :, : ]

            sdr, _, _, _ = separation.bss_eval_sources(reference_sources, estimated_sources)

            sdr_loss += sdr.mean()
        sdr_loss /= output_batch.shape[0]

        optimizer.zero_grad()
#         loss.backward()
        optimizer.step()

#         total_loss += loss.item()

    print(f"Epoch {epoch+1}, Average SDR Loss: {sdr_loss/len(data_loader)}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 118.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 41.06 MiB is free. Process 2733 has 14.71 GiB memory in use. Of the allocated memory 14.30 GiB is allocated by PyTorch, and 276.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF